In [36]:
cups = list(map(int, '538914762'))
#cups = list(map(int, '389125467'))
m = max(cups)
current = 0
for _ in range(100):
    current_value = cups[current]
    # print('---')
    # print(f"cups: {cups}\ncurrent: {current_value}")
    pick_up = []
    for _ in range(3):
        pick_up.append(cups.pop((current+1)%len(cups)))
        current = cups.index(current_value)
    destination_value = 1 + (current_value-2)%m
    while destination_value not in cups:
        destination_value = 1+(destination_value-2)%m
    destination = cups.index(destination_value)
    cups = cups[:destination+1] + pick_up + cups[destination+1:]
    current = cups.index(current_value)
    
    # print(f"pick up: {pick_up}\ndestination: {destination_value}")
    
    current = (current+1)%len(cups)

In [37]:
one = cups.index(1)
print(''.join(map(str, cups[one+1:] + cups[:one])))

54327968


In [107]:
class DoublyLinkedIterator:
    def __init__(self, start, values=True):
        self.start = start
        self.current = start
        self.started = False
        self.values = values
    
    def __next__(self):
        if self.current is None or (self.started and self.current == self.start):
            raise StopIteration
        self.started = True
        v = self.current.value if self.values else self.current
        self.current = self.current._next
        return v

class DoublyLinkedNodesIterable:
    def __init__(self, node):
        self.node = node
    
    def __iter__(self):
        return DoublyLinkedIterator(self.node, False)
    
class DoublyLinkedNode:
    def __init__(self, value):
        self.value = value
        self._prev = None
        self._next = None
    
    def __iter__(self):
        return DoublyLinkedIterator(self)
    
    def __str__(self):
        return f"({' -> '.join(map(str, self))})"
    
    def __repr__(self):
        return f"DoublyLinkedNode{self}"
    
    def set_next(self, n):
        old_next = self._next
        if old_next is not None:
            self._next = None
            old_next.set_prev(None)
        if n is not None:
            n.set_prev(None)
            n._prev = self
        self._next = n
    
    def set_prev(self, p):
        old_prev = self._prev
        if old_prev is not None:
            self._prev = None
            old_prev.set_next(None)
        if p is not None:
            p.set_next(None)
            p._next = self
        self._prev = p
    
    def __getitem__(self, i):
        a = self
        if i < 0:
            for _ in range(-i):
                a = a._prev
        elif i > 0:
            for _ in range(i):
                a = a._next
        return a
    
    def pop(self, f, t=None):
        if t is None:
            t = 0
        else:
            t -= f + 1
        a = self[f]
        prev = a._prev
        a.set_prev(None)
        last = a[t]
        if prev is not None:
            prev.set_next(last._next)
        else:
            last.set_next(None)
        return a
    
    def nodes(self):
        return DoublyLinkedNodesIterable(self)
    
    def insert_after(self, value):
        assert value != self, "can't insert a containing element"
        n = self._next
        self.set_next(value)
        if n is not None:
            c = value
            while c._next is not None:
                c = c._next
                assert c != self, "can't insert a containing element (state corrupted)"
            c.set_next(n)
            
    def __contains__(self, value):
        if isinstance(value, DoublyLinkedNode):
            value = value.value
        for v in self:
            if v == value:
                return True
        return False

In [119]:
cups = list(map(int, '538914762'))

current = DoublyLinkedNode(cups[0])
prev = current
for v in cups[1:]:
    a = DoublyLinkedNode(v)
    prev.set_next(a)
    prev = a
    
v = max(cups)
for v in range(len(cups), 1000000):
    v += 1
    a = DoublyLinkedNode(v)
    prev.set_next(a)
    prev = a

prev.set_next(current)
    
nodes = {}
for n in current.nodes():
    nodes[n.value] = n

m = max(nodes)

for i in range(10000000):
    if i % 1000000 == 0:
        print(i)
    # print("---")
    # print(current)
    pick_up = current.pop(1, 4)
    # print(pick_up)
    destination_value = 1+(current.value - 2)%m
    while destination_value in pick_up:
        destination_value = 1+(destination_value - 2)%m
    nodes[destination_value].insert_after(pick_up)
    current = current._next
    
a = nodes[1][1].value
b = nodes[1][2].value
print(a, b, a*b)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
870999 180724 157410423276
